In [32]:
import random
from datetime import datetime
from faker import Faker

fake = Faker()

def gen_fake_body(paragraph_len=5, paragraphs=5):
    para = ""
    for i in range(random.randint(2, paragraph_len)):
        for j in range(random.randint(1, paragraphs)):
            para += fake.paragraph()
            para += "\\n"
        para += "\\n\\n"
    return para

def generate_users(user_amt):
    """
    Generate Users
    
    CREATE TABLE `users` (
        `id` integer PRIMARY KEY AUTO_INCREMENT,
        `username` varchar(255) UNIQUE,
        `email` varchar(255) UNIQUE,
        `password` varchar(255) UNIQUE,
        `first_name` varchar(80),
        `last_name` varchar(80),
        `created_at` timestamp DEFAULT (now())
    );
    """
    users = []
    for i in range(user_amt):
        username = fake.user_name()
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = f"{first_name}.{last_name}@{fake.domain_name()}"
        password = fake.password(10, False, False, False)
        created_at = fake.date_time_this_year()
        users.append([i, username, email, password, first_name, last_name, created_at])
    return users

def generate_posts(post_amt, user_amt, users):
    """
    Generate Posts
    
    CREATE TABLE `posts` (
        `id` integer PRIMARY KEY AUTO_INCREMENT,
        `title` varchar(255),
        `body` text COMMENT 'Content of the post',
        `user_id` integer,
        `created_at` timestamp DEFAULT (now()),
        `updated_times` integer,
        `updated_at` timestamp DEFAULT (now()),
        FOREIGN KEY (`user_id`) REFERENCES `users` (`id`)
    );
    """
    posts = []
    for i in range(post_amt):
        title = fake.sentence()
        body = gen_fake_body(20, 10)
        user_id = random.randint(1, user_amt)
        user_created = users[user_id - 1][6]
        created_at = fake.date_time_between_dates(user_created, datetime.now())
        updated_times = 1 if random.random() > 0.9 else 0
        updated_at = fake.date_time_between_dates(created_at, datetime.now()) if updated_times > 0 else None
        posts.append([i, title, body, user_id, created_at, updated_times, updated_at])
    return posts
        
def generate_comments(comment_amt, post_amt, posts):
    """
    Generate Comments
    
    CREATE TABLE `comments` (
        `id` integer PRIMARY KEY AUTO_INCREMENT,
        `user_id` integer,
        `post_id` integer,
        `reply_to` integer,
        `body` text COMMENT 'Content of the comment',
        `created_at` timestamp DEFAULT (now()),
        `updated_times` integer,
        `updated_at` timestamp DEFAULT (now()),
        FOREIGN KEY (`user_id`) REFERENCES `users` (`id`),
        FOREIGN KEY (`post_id`) REFERENCES `posts` (`id`),
        FOREIGN KEY (`reply_to`) REFERENCES `comments` (`id`)
    );
    """
    def find_reply_to(post_id, comments):
        reply_to = None
        if random.random() > 0.6:
            for comment in comments:
                if comment[1] == posts[post_id - 1][0]:
                    if reply_to is not None:
                        if random.random() > 0.5:
                            reply_to = comment[0]+1
                    else:
                        reply_to = comment[0]+1
        return reply_to
        
    comments = []
    for i in range(comment_amt):
        post_id = random.randint(1, post_amt)
        reply_to = find_reply_to(post_id, comments)
        post_created = posts[post_id - 1][4]
        if reply_to is not None:
            post_created = comments[reply_to - 1][4]
        body = gen_fake_body(5, 5)
        created_at = fake.date_time_between_dates(post_created, datetime.now())
        updated_times = 1 if random.random() > 0.85 else 0
        updated_at = fake.date_time_between_dates(created_at, datetime.now()) if updated_times > 0 else None
        comments.append([i, post_id, reply_to, body, created_at, updated_times, updated_at])
    return comments
        
def generate_likes(like_amt, post_amt, user_amt, posts, users):
    """
    Generate Likes
    
    CREATE TABLE `likes` (
        `post_id` integer,
        `user_id` integer,
        `liked_at` timestamp DEFAULT (now()),
        PRIMARY KEY (`post_id`, `user_id`),
        FOREIGN KEY (`post_id`) REFERENCES `posts` (`id`),
        FOREIGN KEY (`user_id`) REFERENCES `users` (`id`)
    );
    """
    likes = []
    liked_pairs = set() # to keep track of which post/user pairs have already been liked
    while len(likes) < like_amt:
        post_id = random.randint(1, post_amt)
        post_created = posts[post_id - 1][4]
        user_id = random.randint(1, user_amt)
        user_created = users[user_id - 1][6]
        if user_created < post_created:
            liked_pair = (post_id, user_id)
            if liked_pair not in liked_pairs:
                liked_pairs.add(liked_pair)
                liked_at = fake.date_time_between_dates(post_created, datetime.now())
                likes.append([post_id, user_id, liked_at])
    return likes

def generate_follows(follow_amt, users):
    """Generate random follows

    CREATE TABLE `follows` (
        `following_user_id` integer,
        `followed_user_id` integer,
        `created_at` timestamp DEFAULT (now()),
        PRIMARY KEY (`following_user_id`, `followed_user_id`),
        FOREIGN KEY (`following_user_id`) REFERENCES `users` (`id`),
        FOREIGN KEY (`followed_user_id`) REFERENCES `users` (`id`)
    );
    """
    follows = []
    unique_follows = set() # to keep track of which post/user pairs have already been liked
    while len(follows) < follow_amt:
        following_user_id = random.randint(1, user_amt)
        followed_user_id = random.randint(1, user_amt)
        if following_user_id != followed_user_id:
            following_user_created = users[following_user_id-1][6]
            followed_user_created = users[followed_user_id-1][6]
            unique_follow = (following_user_id, followed_user_id)
            if unique_follow not in unique_follows:
                unique_follows.add(unique_follow)
                created_at = fake.date_time_between_dates(max([following_user_created, followed_user_created]), datetime.now())
                follows.append([following_user_id, followed_user_id, created_at])
    return follows

user_amt = 30
post_amt = user_amt * 4
comment_amt = post_amt * 2
like_amt = post_amt * 5
follow_amt = user_amt * 5

users = generate_users(user_amt)
posts = generate_posts(post_amt, user_amt, users)
comments = generate_comments(comment_amt, post_amt, posts)
likes = generate_likes(like_amt, post_amt, user_amt, posts, users)
follows = generate_follows(follow_amt, users)

for i in range(user_amt):
    users[i][6] = users[i][6].strftime('%Y-%m-%d %H:%M:%S')
for i in range(post_amt):
    posts[i][4] = posts[i][4].strftime('%Y-%m-%d %H:%M:%S')
    if posts[i][6] is not None:
        posts[i][6] = posts[i][6].strftime('%Y-%m-%d %H:%M:%S')
for i in range(comment_amt):
    comments[i][4] = comments[i][4].strftime('%Y-%m-%d %H:%M:%S')
    if comments[i][6] is not None:
        comments[i][6] = comments[i][6].strftime('%Y-%m-%d %H:%M:%S')
for i in range(like_amt):
    likes[i][2] = likes[i][2].strftime('%Y-%m-%d %H:%M:%S')
for i in range(follow_amt):
    follows[i][2] = follows[i][2].strftime('%Y-%m-%d %H:%M:%S')
    
# Prepare the SQL insert statements
sql_statements = open("drop_tables.sql", "r+").readlines()
sql_statements += "\n"
sql_statements += open("first.sql", "r+").readlines()
sql_statements += "\n\n"
# Insert users
for user in users:
    sql_statements.append(f"INSERT INTO users (username, email, password, first_name, last_name, created_at) VALUES ('{user[1]}', '{user[2]}', '{user[3]}', '{user[4]}', '{user[5]}', '{user[6]}');\n")

# Insert posts
for post in posts:
    updated_at_sql = f"'{post[6]}'" if post[6] is not None else 'NULL'
    sql_statements.append(f"INSERT INTO posts (title, body, user_id, created_at, updated_times, updated_at) VALUES ('{post[1]}', '{post[2]}', {post[3]}, '{post[4]}', {post[5]}, {updated_at_sql});\n")

# Insert comments
for comment in comments:
    reply_to_sql = f"'{comment[2]}'" if comment[2] is not None else 'NULL'
    updated_at_sql = f"'{comment[6]}'" if comment[6] is not None else 'NULL'
    sql_statements.append(f"INSERT INTO comments (post_id, reply_to, body, created_at, updated_times, updated_at) VALUES ({comment[1]}, {reply_to_sql}, '{comment[3]}', '{comment[4]}', {comment[5]}, {updated_at_sql});\n")

# Insert likes
for like in likes:
    sql_statements.append(f"INSERT INTO likes (post_id, user_id, liked_at) VALUES ({like[0]}, {like[1]}, '{like[2]}');\n")

# Insert follows
for follow in follows:
    sql_statements.append(f"INSERT INTO follows (following_user_id, followed_user_id, created_at) VALUES ({follow[0]}, {follow[1]}, '{follow[2]}');\n")
    
# Write SQL statements to a file
with open('createDB.sql', 'w+') as sql_file:
    for statement in sql_statements:
        sql_file.write(statement)